In [23]:
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.2f' % x)

import numpy as np

from bokeh.plotting import figure, output_file, show
from bokeh.models import DatetimeTickFormatter
from bokeh.io import output_file, show
from bokeh.layouts import row
from bokeh.layouts import column
from bokeh.layouts import gridplot
from bokeh.models import Button
from bokeh.palettes import RdYlBu3
from bokeh.plotting import figure, curdoc
from bokeh.layouts import widgetbox
from bokeh.models.widgets import MultiSelect,DataTable,TableColumn,NumberFormatter,Slider,Panel,Tabs,Paragraph,Div,Dropdown
from bokeh.models import LinearAxis,Range1d,FactorRange,ColumnDataSource

In [431]:
#df=sf.copy()
df1=ic.copy()

In [432]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7068 entries, 0 to 7067
Data columns (total 7 columns):
DBA_NAME       7068 non-null object
COST_MONTH     7068 non-null datetime64[ns]
CARD_SCHEME    7068 non-null object
VOLUME         7068 non-null int64
IC             7068 non-null float64
FLOW           7068 non-null float64
PLATFORM       7068 non-null object
dtypes: datetime64[ns](1), float64(2), int64(1), object(3)
memory usage: 386.6+ KB


In [455]:
#df_working=df.copy()
df1_working=df1.copy()

In [456]:
#df_working=df_working[['COST_MONTH','CARD_SCHEME','VOLUME','SF','FLOW','PLATFORM']]
df1_working=df1_working[['COST_MONTH','CARD_SCHEME','VOLUME','IC','FLOW','PLATFORM']]
df1_working_rank=df1[['DBA_NAME','COST_MONTH','CARD_SCHEME','VOLUME','IC','FLOW','PLATFORM']]

In [457]:
#df_working=df_working.groupby(['COST_MONTH','CARD_SCHEME','PLATFORM'],as_index=False).sum()
df1_working=df1_working.groupby(['COST_MONTH','CARD_SCHEME','PLATFORM'],as_index=False).sum()

In [458]:
#df_working['SF_BPS']=df_working['SF']/df_working['FLOW']
df1_working['IC_BPS']=df1_working['IC']/df1_working['FLOW']

In [459]:
#IC fee conditions
df1_working_vusa=df1_working[(df1_working['PLATFORM']=='WF USA')&(df1_working['CARD_SCHEME']=='VISA')]
df1_working_vcad=df1_working[(df1_working['PLATFORM']=='WF CANADA')&(df1_working['CARD_SCHEME']=='VISA')]
df1_working_mcusa=df1_working[(df1_working['PLATFORM']=='WF USA')&(df1_working['CARD_SCHEME']=='MASTERCARD')]
df1_working_mccad=df1_working[(df1_working['PLATFORM']=='WF CANADA')&(df1_working['CARD_SCHEME']=='MASTERCARD')]
df1_working_dusa=df1_working[(df1_working['PLATFORM']=='WF USA')&(df1_working['CARD_SCHEME']=='DISCOVER')]
df1_working_dcad=df1_working[(df1_working['PLATFORM']=='WF CANADA')&(df1_working['CARD_SCHEME']=='DISCOVER')]

In [460]:
#IC ranking
df1_working_rank_vusa=df1_working_rank[(df1_working_rank['PLATFORM']=='WF USA')&(df1_working_rank['CARD_SCHEME']=='VISA')]
df1_working_rank_vusa=df1_working_rank_vusa[['DBA_NAME','FLOW']]
df1_working_rank_vusa=df1_working_rank_vusa.groupby(['DBA_NAME'],as_index=False).sum()
df1_working_rank_vusa['RANK']=(df1_working_rank_vusa['FLOW']/df1_working_rank_vusa['FLOW'].sum())

df1_working_rank_vusa=df1_working_rank_vusa.sort_values(by='RANK',ascending=False)
df1_working_rank_vusa=df1_working_rank_vusa[:10]
df1_working_rank_vusa=df1_working_rank_vusa.reset_index()
df1_working_rank_vusa=df1_working_rank_vusa.reset_index()
df1_working_rank_vusa=df1_working_rank_vusa.drop('index',axis=1)
df1_working_rank_vusa=df1_working_rank_vusa[['level_0','RANK','DBA_NAME']]
#==========================================================================================================================
df1_working_vusa_tbl=pd.merge(df1_working_rank_vusa,df1,on='DBA_NAME',how='left')

In [461]:
df1_working_vusa_tbl.head()

,level_0,RANK,DBA_NAME,COST_MONTH,CARD_SCHEME,VOLUME,IC,FLOW,PLATFORM
0,0,0.34,BLIZZARD,2018-02-01,MASTERCARD,390133,206489.93,8998288.20,WF USA
1,0,0.34,BLIZZARD,2018-06-01,MASTERCARD,300555,131203.31,5526461.43,WF USA
2,0,0.34,BLIZZARD,2018-04-01,VISA,1079984,430264.58,20891794.62,WF USA
3,0,0.34,BLIZZARD,2018-02-01,VISA,1263082,644030.65,28853792.19,WF USA
4,0,0.34,BLIZZARD,2018-05-01,VISA,1090348,418087.68,20525748.13,WF USA


In [462]:
df1_working_vusa_tbl=df1_working_vusa_tbl[['DBA_NAME','level_0','RANK','COST_MONTH','IC','FLOW','CARD_SCHEME']]
df1_working_vusa_tbl=df1_working_vusa_tbl[(df1_working_vusa_tbl['CARD_SCHEME']=='VISA')]
df1_working_vusa_tbl['IC_BPS']=(df1_working_vusa_tbl['IC']/df1_working_vusa_tbl['FLOW'])*10000
df1_working_vusa_tbl=pd.pivot_table(df1_working_vusa_tbl, values=['IC_BPS'], index=['level_0','RANK','DBA_NAME'], columns='COST_MONTH', aggfunc='mean', fill_value='', margins=False, dropna=True, margins_name='All')

In [463]:
df1_working_vusa_tbl

IC_BPS                                   \
COST_MONTH                        2017-07-01 2017-08-01 2017-09-01 2017-10-01   
level_0 RANK DBA_NAME                                                           
0       0.34 BLIZZARD                                       220.27     241.04   
1       0.30 VIATOR/TRIPADVISOR                                                 
2       0.05 AE.COM WEB PURCH CAD     320.49     322.17     318.99     358.42   
3       0.04 SQUARE ENIX                         257.24     261.67     202.24   
4       0.03 PRICELINE.COMUSD                                                   
5       0.03 PLN*PRICELINE HOTELS                                               
6       0.03 BLIZZARD CAD                                                       
7       0.02 UFC.TV                   183.08     185.77     227.06     221.55   
8       0.02 ROSETTA STONE            272.75     283.70     276.23     281.54   
9       0.01 ELSEVIER USD ECOMM       260.09     212.97     205.19     238.91   

                                                                               \
COST_MONTH                        2017-11-01 2017-12-01 2018-01-01 2018-02-01   
level_0 RANK DBA_NAME                                                           
0       0.34 BLIZZARD                 251.96     245.84     221.09     223.20   
1       0.30 VIATOR/TRIPADVISOR                  324.47     256.97     302.04   
2       0.05 AE.COM WEB PURCH CAD     325.26     385.96     255.60     279.82   
3       0.04 SQUARE ENIX              257.19     242.79     243.01     263.20   
4       0.03 PRICELINE.COMUSD                    286.50     287.03     302.82   
5       0.03 PLN*PRICELINE HOTELS                258.60     264.20     269.80   
6       0.03 BLIZZARD CAD             313.79     350.24     251.70     359.41   
7       0.02 UFC.TV                   190.57     205.57     186.53     224.91   
8       0.02 ROSETTA STONE            282.31     207.71     182.83     176.01   
9       0.01 ELSEVIER USD ECOMM       285.14     295.71     281.15     280.60   

                                                                               
COST_MONTH                        2018-03-01 2018-04-01 2018-05-01 2018-06-01  
level_0 RANK DBA_NAME                                                          
0       0.34 BLIZZARD                 208.12     205.95     203.69     206.16  
1       0.30 VIATOR/TRIPADVISOR       254.61     256.12     246.20     242.67  
2       0.05 AE.COM WEB PURCH CAD     259.06     256.82     248.38     245.23  
3       0.04 SQUARE ENIX              252.43     234.52     227.15     226.89  
4       0.03 PRICELINE.COMUSD         291.41     292.42     288.56     275.95  
5       0.03 PLN*PRICELINE HOTELS     268.38     265.85     256.58     249.48  
6       0.03 BLIZZARD CAD             254.97     253.71     244.15     241.60  
7       0.02 UFC.TV                   217.64     201.92     218.72     191.39  
8       0.02 ROSETTA STONE            172.40     173.02     163.03     163.26  
9       0.01 ELSEVIER USD ECOMM       283.70     291.66     270.07     272.05

In [464]:
df1_working_vusa_tbl=pd.DataFrame(df1_working_vusa_tbl.to_records())
df1_working_vusa_tbl=df1_working_vusa_tbl.rename(columns={'level_0':'Ranking','RANK':'Percent of Flow','DBA_NAME':'Merchant'})

In [465]:
df1_working_vusa_tbl_1=df1_working_vusa_tbl.drop(['Ranking','Percent of Flow','Merchant'],axis=1)

In [466]:
curdoc().clear()
x1=['2017-09-01','2017-09-01','2017-09-01','2017-10-01','2017-11-01','2017-12-01','2018-01-01','2018-02-01','2018-03-01',
  '2018-04-01','2018-05-01','2018-06-01']
x2=['2017-11-01','2017-11-01','2017-11-01','2017-11-01','2017-11-01','2017-12-01','2018-01-01','2018-02-01','2018-03-01',
  '2018-04-01','2018-05-01','2018-06-01']
x3=['2017-07-01','2017-08-01','2017-09-01','2017-10-01','2017-11-01','2017-12-01','2018-01-01','2018-02-01','2018-03-01',
  '2018-04-01','2018-05-01','2018-06-01']
x4=['2017-08-01','2017-08-01','2017-09-01','2017-10-01','2017-11-01','2017-12-01','2018-01-01','2018-02-01','2018-03-01',
  '2018-04-01','2018-05-01','2018-06-01']
x5=['2017-12-01','2017-12-01','2017-12-01','2017-12-01','2017-12-01','2017-12-01','2018-01-01','2018-02-01','2018-03-01',
  '2018-04-01','2018-05-01','2018-06-01']
x6=['2017-12-01','2017-12-01','2017-12-01','2017-12-01','2017-12-01','2017-12-01','2018-01-01','2018-02-01','2018-03-01',
  '2018-04-01','2018-05-01','2018-06-01']
x7=['2017-11-01','2017-11-01','2017-11-01','2017-11-01','2017-11-01','2017-12-01','2018-01-01','2018-02-01','2018-03-01',
  '2018-04-01','2018-05-01','2018-06-01']
x8=['2017-07-01','2017-08-01','2017-09-01','2017-10-01','2017-11-01','2017-12-01','2018-01-01','2018-02-01','2018-03-01',
  '2018-04-01','2018-05-01','2018-06-01']
x9=['2017-07-01','2017-08-01','2017-09-01','2017-10-01','2017-11-01','2017-12-01','2018-01-01','2018-02-01','2018-03-01',
  '2018-04-01','2018-05-01','2018-06-01']
x10=['2017-07-01','2017-08-01','2017-09-01','2017-10-01','2017-11-01','2017-12-01','2018-01-01','2018-02-01','2018-03-01',
  '2018-04-01','2018-05-01','2018-06-01']


y1=df1_working_vusa_tbl_1.iloc[0].values.tolist()
y1[0]=220.2669800921614
y1[1]=220.2669800921614
#y2=df1_working_vusa_tbl_1.iloc[1].values.tolist()
#y2[0]
#y2[1]
#y2[3]
#y2[0]
y3=df1_working_vusa_tbl_1.iloc[2].values.tolist()
y4=df1_working_vusa_tbl_1.iloc[3].values.tolist()
y4[0]=257.24292638197534
y5=df1_working_vusa_tbl_1.iloc[4].values.tolist()
y5[0]=286.5043078588005
y5[1]=286.5043078588005
y5[2]=286.5043078588005
y5[3]=286.5043078588005
y5[4]=286.5043078588005
y6=df1_working_vusa_tbl_1.iloc[5].values.tolist()
y6[0]=258.60152779931315
y6[1]=258.60152779931315
y6[2]=258.60152779931315
y6[3]=258.60152779931315
y6[4]=258.60152779931315
y7=df1_working_vusa_tbl_1.iloc[6].values.tolist()
y7[0]=313.7917741406992
y7[1]=313.7917741406992
y7[2]=313.7917741406992
y7[3]=313.7917741406992
y8=df1_working_vusa_tbl_1.iloc[7].values.tolist()

x1=x1
x3=x3
x4=x4
x5=x5
x6=x6
x7=x7
x8=x8

#converting x-axis to datetime
x1 = [pd.datetime.strptime(date, "%Y-%m-%d") for date in x1]
#x2 = [pd.datetime.strptime(date, "%Y-%m-%d") for date in x2]
x3 = [pd.datetime.strptime(date, "%Y-%m-%d") for date in x3]
x4 = [pd.datetime.strptime(date, "%Y-%m-%d") for date in x4]
x5 = [pd.datetime.strptime(date, "%Y-%m-%d") for date in x5]
x6 = [pd.datetime.strptime(date, "%Y-%m-%d") for date in x6]
x7 = [pd.datetime.strptime(date, "%Y-%m-%d") for date in x7]
x8 = [pd.datetime.strptime(date, "%Y-%m-%d") for date in x8]


p = figure(title="Top 10 Merchant Trends", x_axis_label='date', y_axis_label='Basis Point', x_axis_type='datetime'
           ,plot_width=600, plot_height=700)

p.line(x=x1,y=y1,legend='Blizzard',color='blue',line_width=3)
p.line(x=x3,y=y3,legend='American Eagle Canada',color='green',line_width=3)
p.line(x=x4,y=y4,legend='Square Enix',color='red',line_width=3)
p.line(x=x5,y=y5,legend='Priceline',color='purple',line_width=3)
p.line(x=x6,y=y6,legend='Priceline Hotels',color='black',line_width=3)
p.line(x=x7,y=y7,legend='Blizzard Canada',color='orange',line_width=3)
p.line(x=x8,y=y8,legend='UFC',color='deeppink',line_width=3)

p.legend.location='bottom_left'
p.legend.click_policy='hide'


#show(p)

In [467]:
data = df1_working_vusa_tbl
source = ColumnDataSource(data)

columns = [
TableColumn(field="Ranking", title="Ranking (by Flow)"),
TableColumn(field="Percent of Flow", title="Percent of Flow",formatter=NumberFormatter(format="0.0%")),
TableColumn(field="Merchant", title="Merchant"),
TableColumn(field="('IC_BPS', Timestamp('2017-07-01 00:00:00'))", title="07-2017",formatter=NumberFormatter(format="0.0")),
TableColumn(field="('IC_BPS', Timestamp('2017-08-01 00:00:00'))", title="08-2017",formatter=NumberFormatter(format="0.0")),
TableColumn(field="('IC_BPS', Timestamp('2017-09-01 00:00:00'))", title="09-2017",formatter=NumberFormatter(format="0.0")),
TableColumn(field="('IC_BPS', Timestamp('2017-10-01 00:00:00'))", title="10-2017",formatter=NumberFormatter(format="0.0")),
TableColumn(field="('IC_BPS', Timestamp('2017-11-01 00:00:00'))", title="11-2017",formatter=NumberFormatter(format="0.0")),
TableColumn(field="('IC_BPS', Timestamp('2017-12-01 00:00:00'))", title="12-2017",formatter=NumberFormatter(format="0.0")),
TableColumn(field="('IC_BPS', Timestamp('2018-01-01 00:00:00'))", title="01-2018",formatter=NumberFormatter(format="0.0")),
TableColumn(field="('IC_BPS', Timestamp('2018-02-01 00:00:00'))", title="02-2018",formatter=NumberFormatter(format="0.0")),
TableColumn(field="('IC_BPS', Timestamp('2018-03-01 00:00:00'))", title="03-2018",formatter=NumberFormatter(format="0.0")),
TableColumn(field="('IC_BPS', Timestamp('2018-04-01 00:00:00'))", title="04-2018",formatter=NumberFormatter(format="0.0")),
TableColumn(field="('IC_BPS', Timestamp('2018-05-01 00:00:00'))", title="05-2018",formatter=NumberFormatter(format="0.0")),
TableColumn(field="('IC_BPS', Timestamp('2018-06-01 00:00:00'))", title="06-2018",formatter=NumberFormatter(format="0.0")),
]
data_table1 = DataTable(source=source,columns=columns, width=1700, height=600,fit_columns=True)

div = Div(text="""Your <a href="https://en.wikipedia.org/wiki/HTML">HTML</a>-supported text is initialized with the <b>text</b> argument.  The
remaining div arguments are <b>width</b> and <b>height</b>. For this example, those values
are <i>200</i> and <i>100</i> respectively.""",
width=200, height=100)
top=row(p6,p)
grid4=column(widgetbox(div),top,widgetbox(data_table1))

In [468]:
df1_rolling_vusa=df1_working_vusa[['COST_MONTH','IC_BPS']]
df1_rolling_vcad=df1_working_vcad[['COST_MONTH','IC_BPS']]
df1_rolling_mcusa=df1_working_mcusa[['COST_MONTH','IC_BPS']]
df1_rolling_mccad=df1_working_mccad[['COST_MONTH','IC_BPS']]
df1_rolling_dusa=df1_working_dusa[['COST_MONTH','IC_BPS']]
df1_rolling_dcad=df1_working_dcad[['COST_MONTH','IC_BPS']]

In [469]:
df1_rolling_vusa=df1_working_vusa.IC_BPS.rolling(window=3,min_periods=3).mean()
df1_rolling_vcad=df1_working_vcad.IC_BPS.rolling(window=3,min_periods=3).mean()
df1_rolling_mcusa=df1_working_mcusa.IC_BPS.rolling(window=3,min_periods=3).mean()
df1_rolling_mccad=df1_working_mccad.IC_BPS.rolling(window=3,min_periods=3).mean()
df1_rolling_dusa=df1_working_dusa.IC_BPS.rolling(window=3,min_periods=3).mean()
df1_rolling_dcad=df1_working_dcad.IC_BPS.rolling(window=3,min_periods=3).mean()

In [470]:
df1_rolling_vusa=df1_rolling_vusa*10000
df1_rolling_vcad=df1_rolling_vcad*10000
df1_rolling_mcusa=df1_rolling_mcusa*10000
df1_rolling_mccad=df1_rolling_mccad*10000
df1_rolling_dusa=df1_rolling_dusa*10000
df1_rolling_dcad=df1_rolling_dcad*10000

In [471]:
curdoc().clear() #since running script in Jupyter and asking to run in html the sessions can duplicate since it's not on a server but in the test environment this script clears the cache clears the document and prevents the duplication error
# prepare some data
xv=df1_working_vusa['COST_MONTH']
yv=df1_working_vusa['IC_BPS']*10000
wv=df1_rolling_vusa

# output to static HTML file
output_file("lines.html")


# create a new plot with a title and axis labels
p6 = figure(title="Wells Fargo USA Visa IC Fees", x_axis_label='Date', y_axis_label='Basis Points', x_axis_type='datetime',\
           plot_width=600, plot_height=500)

# add a line renderer with legend and line thickness
p6.line(xv,yv,legend="Basis Points", line_width=3)
p6.line(xv,wv,legend="3-Month Rolling Avg.", line_width=2,line_color='black',line_dash='dashed')
p6.circle(xv,yv)

#===================================================================================================

# prepare some data
xc=df1_working_vcad['COST_MONTH']
yc=df1_working_vcad['IC_BPS']*10000
wc=df1_rolling_vcad

p7 = figure(title="Wells Fargo Canada Visa IC Fees", x_axis_label='Date', y_axis_label='Basis Points', x_axis_type='datetime',\
           plot_width=600, plot_height=500)

# add a line renderer with legend and line thickness
p7.line(xc,yc,legend="Basis Points", line_width=3)
p7.line(xv,wc,legend="3-Month Rolling Avg.", line_width=2,line_color='black',line_dash='dashed')
p7.circle(xc,yc)

#===================================================================================================

# prepare some data
xmu=df1_working_mcusa['COST_MONTH']
ymu=df1_working_mcusa['IC_BPS']*10000
wmu=df1_rolling_mcusa

p8 = figure(title="Wells Fargo USA MasterCard IC Fees", x_axis_label='Date', y_axis_label='Basis Points', x_axis_type='datetime',\
           plot_width=600, plot_height=500)

# add a line renderer with legend and line thickness
p8.line(xmu,ymu,legend="Basis Points", line_width=3,color="orange")
p8.line(xv,wmu,legend="3-Month Rolling Avg.", line_width=2,line_color='black',line_dash='dashed')
p8.circle(xmu,ymu,color="orange")

#====================================================================================================
# prepare some data
xmc=df1_working_mccad['COST_MONTH']
ymc=df1_working_mccad['IC_BPS']*10000
wmc=df1_rolling_mccad

p9= figure(title="Wells Fargo Canada MasterCard IC Fees", x_axis_label='Date', y_axis_label='Basis Points', x_axis_type='datetime',\
           plot_width=600, plot_height=500)

# add a line renderer with legend and line thickness
p9.line(xmc,ymc,legend="Basis Points", line_width=3,color="orange")
p9.line(xv,wmc,legend="3-Month Rolling Avg.", line_width=2,line_color='black',line_dash='dashed')
p9.circle(xmc,ymc,color="orange")

#====================================================================================================
# prepare some data
xdu=df1_working_dusa['COST_MONTH']
ydu=df1_working_dusa['IC_BPS']*10000
wdu=df1_rolling_dusa

p10=figure(title="Wells Fargo USA Discover IC Fees", x_axis_label='Date', y_axis_label='Basis Points', x_axis_type='datetime',\
           plot_width=600, plot_height=500)

# add a line renderer with legend and line thickness
p10.line(xdu,ydu,legend="Basis Points", line_width=3,color="black")
p10.line(xv,wdu,legend="3-Month Rolling Avg.", line_width=2,line_color='black',line_dash='dashed')
p10.circle(xdu,ydu,color="black")

#=====================================================================================================


#====================================================================================================
grid2 = gridplot([[p6, p8,p10], [p7,p9,None]])
#show(grid)

In [472]:
#tab1=Panel(child=grid1,title='Wells Fargo Scheme Fees')
tab2=Panel(child=grid2,title='Wells Fargo Interchange Fees')
#tab3=Panel(child=grid3,title='Wells Fargo Scheme + Interchange Fees')
tab4=Panel(child=grid4,title='Tables')

#tabs = Tabs(tabs=[tab1,tab2,tab3])
tabs = Tabs(tabs=[tab2,tab4])

In [473]:
show(tabs)